### Optimization of a radiative cooling structure
This notebook will optimize a multi-layer structure inspired by Raman  𝑒𝑡   𝑎𝑙. , Nature 515, 540-546 (2014), utilizing analytic gradients of the three terms that dominate
the radiative flux: absorption of solar radiation, absorption of atmospheric thermal radiation (both warming), and thermal emission (cooling).  

**Note that explit angle and polarization dependence will be included in these gradients, and so each gradient evaluation will be ~14 times slower than comparable gradient evaluations when angle- and polarization dependence are ignored as in other optimization demos [see, for example, here](http://localhost:8888/notebooks/CODE/wptherml/example/Incandescent_Opt.ipynb)**.

In [2]:
### Import WPTHERML class!
from wptherml.wpml import multilayer
from matplotlib import pyplot as plt
from wptherml.datalib import datalib
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import time
import numpy as np

### Define structure!
structure = {

        'Material_List': ['Air', 'SiO2', 'HfO2', 'SiO2', 'HfO2', 'SiO2', 'HfO2', 'SiO2', 'Ag', 'Air'],
        'Thickness_List': [0, 230e-9, 485e-9, 688e-9, 13e-9, 73e-9, 34e-9, 54e-9, 200e-9, 0],
        'Lambda_List': [300e-9, 20000e-9, 1000],
         ### we will compute gradients wrt thickness of first 7 layers, Ag thickness will not change
        'Gradient_List': [1,2,3,4,5,6,7],
        'EXPLICIT_ANGLE': 1,
        'COOLING': 1
     
        }

### create instance of multilayer class called cool_ml
cool_ml = multilayer(structure)
### get length of gradient vector
length = cool_ml.gradient_dimension

def update_multilayer(x):
    for i in range(0,len(x)):
        cool_ml.d[i+1] = x[i]*1e-9
    ### now we have the new structure, update fresnel quantities
    cool_ml.fresnel_ea()
    cool_ml.thermal_emission_ea()
    ### now we have new emissivity, update thermal emission
    cool_ml.cooling_power()

    ### return negative of cooling power - minimize functions want 
    ### to minimize, so trick them by passing negative of the objective you
    ### want to maximize
    return -cool_ml.cooling_power_val

### given an array of thicknesses of the coating, update
### the structure and compute the gradient vector of conversion efficiency wrt layer thicknesses
def analytic_grad(x0):
    cur = update_multilayer(x0)
    cool_ml.fresnel_prime_ea()
    cool_ml.thermal_emission_prime_ea()
    cool_ml.cooling_power_prime()
    ### now there will be three contributions to the
    ### gradient vector... two warming contributions 
    ### (solar_power_grad and atmospheric_power_grad)
    ### and one cooling contribution (radiative_power_grad)
    ### and the total gradient is the sum of all three
    g = cool_ml.cooling_power_grad
    ### scale gradient to be in nm^-1 rather than over m^-1
    return -g*1e-9

### Function that gets the negative of the efficiency and the 
### negative of the gradient for use in the l-bfgs-b algorithm
### also prints out the time for timing purposes!
def SuperFunc(x0):
    en = update_multilayer(x0)
    c_time = time.time()
    if en<0:
        print(" This structure is net cooling with net power out being",-en)
    else:
        print(" This structure is net warming with net poer in being",-en)
    gr = analytic_grad(x0)
    return en, gr


# the bounds for L-BFGS-B updates!
bfgs_xmin = np.ones(length)
bfgs_xmax = 300*np.ones(length)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

### initialize the solution vector xs to be the thicknesses from 
### Raman et al. paper
xs = [230, 485, 688, 13, 73, 34, 54]

### print out initial solution vector and initial efficiency
print("xs is ")
print(xs)
pflux = -update_multilayer(xs)
if pflux>0:
    print(" This structure is net cooling with net power out being",pflux)   
else:
    print(" This structure is net warming with net poer in being",pflux)


### run l-bfgs-b algorithm!
ret = minimize(SuperFunc, xs, method="L-BFGS-B", jac=True, bounds=bfgs_bounds)

### print optimal solution and its efficiency!
print(ret.x)
print(-update_multilayer(ret.x))
                                    


 Temperature not specified!
 Proceeding with default T = 300 K
xs is 
[230, 485, 688, 13, 73, 34, 54]
 This structure is net cooling with net power out being 63.525923254487594
 This structure is net cooling with net power out being 26.93358430958938
 This structure is net cooling with net power out being 26.987684491357246
 This structure is net cooling with net power out being 31.257424595611454
 This structure is net cooling with net power out being 59.933261081356505
 This structure is net cooling with net power out being 68.59349206988524
 This structure is net cooling with net power out being 69.19255314674987
 This structure is net cooling with net power out being 69.53944029636773
 This structure is net cooling with net power out being 69.07278380133678
 This structure is net cooling with net power out being 70.57523013190401
 This structure is net cooling with net power out being 70.49876027675884
 This structure is net cooling with net power out being 70.85380995302663
 This 